In [1]:
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load and prepare data

In [2]:
from PIL import Image
import numpy as np
import cv2
from skimage import transform
from keras.preprocessing.image import img_to_array
import tensorflow as tf
import shutil
import os


imsize = (128,128)
dir_x = 'dataset_sunrgbd/img'
dir_y = 'dataset_sunrgbd/labels'

In [3]:
y_train = []
for archivo in os.listdir(dir_y):
    img = Image.open(os.path.join(dir_y,archivo))
    img = cv2.resize(img_to_array(img), dsize=imsize)
    y_train.append(img)
y_train = np.array(y_train).astype('uint8')
y_train = tf.keras.utils.to_categorical(y_train, dtype='float32')
print(y_train.shape)

(10335, 128, 128, 38)


In [4]:
x_train = []
for di in os.listdir(dir_x):
    for archivo in os.listdir(os.path.join(dir_x,di)):
        img = Image.open(os.path.join(os.path.join(dir_x,di),archivo))
        img = cv2.resize(img_to_array(img), dsize=imsize)
        x_train.append(np.asarray(img))
x_train = np.array(x_train)
x_train = x_train/255.0
print(x_train.shape)

(10335, 128, 128, 3)


In [5]:
x_val = x_train[5335:7835]
x_test = x_train[7835:10335]

y_val = y_train[5335:7835]
y_test = y_train[7835:10335]

x_train = x_train[:5335]
y_train = y_train[:5335]

In [6]:
print(x_train.shape , y_train.shape)
print(x_val.shape , y_val.shape)
print(x_test.shape , y_test.shape)

(5335, 128, 128, 3) (5335, 128, 128, 38)
(2500, 128, 128, 3) (2500, 128, 128, 38)
(2500, 128, 128, 3) (2500, 128, 128, 38)


In [7]:
from segmentation_models import get_preprocessing

BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)


x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)
x_test = preprocess_input(x_test)

Segmentation Models: using `keras` framework.


In [22]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=[0.9, 1.25],
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode="nearest")
val_datagen = ImageDataGenerator()

In [23]:
train_datagen.fit(x_train)
val_datagen.fit(x_val)

In [24]:
train = train_datagen.flow(x_train, y_train, batch_size=8)
validation = val_datagen.flow(x_val, y_val, batch_size=8)

# Create model

In [36]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable
import segmentation_models as sm


sm.set_framework('tf.keras')
sm.framework()


# model = Unet(backbone_name='vgg16', encoder_weights='imagenet', encoder_freeze=True, classes=38, activation='softmax', input_shape=(None, None, 3))

model = Unet(backbone_name='resnet101', encoder_weights='imagenet', encoder_freeze=True, classes=38, activation='softmax', input_shape=(None, None, 3))
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, None, None,  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_68 (ZeroPadding2 (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_68[0][0]          
____________________________________________________________________________________________

# Training the model

## Plot Training accuracy and loss

In [37]:
import matplotlib.pyplot as plt

def plot_acc(history, title="Model Accuracy"):
    """Imprime una gráfica mostrando la accuracy por epoch obtenida en un entrenamiento"""
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()
    
def plot_loss(history, title="Model Loss"):
    """Imprime una gráfica mostrando la pérdida por epoch obtenida en un entrenamiento"""
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    plt.show()
    
def plot_compare_losses(history1, history2, name1="Red 1",
                        name2="Red 2", title="Graph title"):
    """Compara losses de dos entrenamientos con nombres name1 y name2"""
    plt.plot(history1.history['loss'], color="green")
    plt.plot(history1.history['val_loss'], 'r--', color="green")
    plt.plot(history2.history['loss'], color="blue")
    plt.plot(history2.history['val_loss'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2],
               loc='upper right')
    plt.show()
    
def plot_compare_accs(history1, history2, name1="Red 1",
                      name2="Red 2", title="Graph title"):
    """Compara accuracies de dos entrenamientos con nombres name1 y name2"""
    plt.plot(history1.history['accuracy'], color="green")
    plt.plot(history1.history['val_accuracy'], 'r--', color="green")
    plt.plot(history2.history['accuracy'], color="blue")
    plt.plot(history2.history['val_accuracy'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2], 
               loc='lower right')
    plt.show()

## metrics

In [38]:
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weight):
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = weight * (logit_y_pred * (1. - y_true) + 
                     K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd
    averaged_mask = K.pool2d(
            y_true, pool_size=(50, 50), strides=(1, 1), padding='same', pool_mode='avg')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight = 5. * K.exp(-5. * K.abs(averaged_mask - 0.5))
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
#seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])

## callbacks

In [39]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import math
weight_path="{}_best_weights.hdf5".format('seg_model_2')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=3, 
                                   verbose=1, min_delta=0.0001, cooldown=2, min_lr=1e-7)

def step_decay(epoch):
    initial_lrate = 1e-5
#     if epoch >= 5:
#         initial_lrate = 5e-5
#     if epoch >= 10:
#         initial_lrate = 2.5e-5
#     if epoch >= 30:
#         initial_lrate = 1e-6
#     if epoch >= 60:
#         initial_lrate = 5e-6
#     print('lr is: ', initial_lrate)
    return initial_lrate
lrate = LearningRateScheduler(step_decay)

early = EarlyStopping(monitor="val_loss",
                      patience=10) # probably needs to be more patient, but kaggle time is limited

callbacks_list = [checkpoint, early, reduceLROnPlat]

## Model fit

In [ ]:
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import precision, iou_score

opt = tf.keras.optimizers.Adam(learning_rate=0.00001, amsgrad=True, name="Adam")

model.compile(optimizer="Adam", loss='categorical_crossentropy', 
              metrics=[dice_coef, 
                       'accuracy', 
                       true_positive_rate, 
                       iou_score
                      ])

history = model.fit(train, epochs=1000, verbose=1, validation_data=validation, callbacks=callbacks_list)

Epoch 1/1000
667/667 [==============================] - 231s 336ms/step - loss: 2.0729 - dice_coef: 0.2101 - accuracy: 0.3967 - true_positive_rate: 0.1275 - iou_score: 0.0245 - val_loss: 2.3737 - val_dice_coef: 0.1381 - val_accuracy: 0.3568 - val_true_positive_rate: 0.0790 - val_iou_score: 0.0190

Epoch 00001: val_loss improved from inf to 2.37367, saving model to seg_model_2_best_weights.hdf5
Epoch 2/1000
667/667 [==============================] - 221s 331ms/step - loss: 1.9298 - dice_coef: 0.2584 - accuracy: 0.4247 - true_positive_rate: 0.1611 - iou_score: 0.0300 - val_loss: 1.8487 - val_dice_coef: 0.2588 - val_accuracy: 0.4561 - val_true_positive_rate: 0.1585 - val_iou_score: 0.0354

Epoch 00002: val_loss improved from 2.37367 to 1.84869, saving model to seg_model_2_best_weights.hdf5
Epoch 3/1000
667/667 [==============================] - 221s 331ms/step - loss: 1.8633 - dice_coef: 0.2820 - accuracy: 0.4410 - true_positive_rate: 0.1796 - iou_score: 0.0337 - val_loss: 1.7591 - val_di

In [ ]:
plot_acc(history)
plot_loss(history)

In [ ]:
score = model.evaluate([x1_test,x2_test], y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
# # Para pasar la segmentacion a imagen
y_pred = model.predict([x1_test,x2_test])
y_classes = np.asarray([np.argmax(y, axis=-1) for y in y_pred])
label = np.asarray([np.argmax(y, axis=-1) for y in y_test])

In [ ]:
number = 3

for i in range (number):
    fig = plt.figure("Labeled Dataset Sample", figsize=(32, 10))
    ax = fig.add_subplot(1, 4, 1)
    plot_color(ax, x1_test[i])
    
    ax = fig.add_subplot(1, 4, 2)
    plot_depth(ax, x2_test[i])

    ax = fig.add_subplot(1, 4, 3)
    plot_label(ax, label[i])

    ax = fig.add_subplot(1, 4, 4)
    plot_label(ax, y_classes[i])

    plt.show()

# Fine tunning

In [ ]:
# release all layers for training
# set all layers trainable and recompile model
for layer in model.layers:
    layer.trainable=True 

In [ ]:
# continue training
history = model.fit(train, epochs=100, verbose=1, validation_data=validation, callbacks=callbacks_list)

In [ ]:
score = model.evaluate([x1_test,x2_test], y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])